In [14]:
import glob, os
from bs4 import BeautifulSoup as bs
import re
import collections
import json
import io
import fix,parser
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        print 'between_error',s,first,last
        return ""

In [32]:
def parse(html,publisher,journal_name):
    global s,s_body
    if publisher=='ama':
        s = bs(html, 'lxml').find('head')
        s_body=bs(html, 'lxml').find('body')
        year_vol_iss= s.find('meta',{'property':'og:description'})['content']
        year=year_vol_iss.split(', Vol')[0][-4:]
        vol= find_between(year_vol_iss,'Vol. ',', No.')
        iss= find_between(year_vol_iss,'No. ',',')
        title=s.find('meta',{'property':'og:title'})['content']
        creators=s.findAll('meta',{'name':'dc.Creator'}); names=[c['content'].strip() for c in creators]
        abstract=s_body.find('div',{'class':'abstractSection'}) or ''
        keywords_html=s.findAll('meta',{'name':'dc.Subject'})
        keywords=', '.join([k['content'] for k in keywords_html])
        url=s.find('meta',{'property':'og:url'})['content']
        if abstract is not '':
            abstract=abstract.find('p').text

    if publisher=='sag':
        s = bs(html, 'lxml').find('head')
        s_body=bs(html, 'lxml').find('body')
        
        try:
            vol_iss_year=s_body.find('div',{"class":"articleJournalNavTitle"}).text.replace('\n','').replace('\t','')
            year=vol_iss_year[-4:]
            iss= vol_iss_year.split('Issue ')[1].split(', ')[0]         
            vol= vol_iss_year.split('Vol ')[1].split(', ')[0]            
        except Exception:
            year=s_body.find('div',{"class":"published-dates"}).text.split('\n')[1][-4:]
            vol='';iss=''
            pass
        
        title=s.find("meta",{"name":"dc.Title"})['content']
        creators=s.findAll("meta",{"name":"dc.Creator"});names=[c['content'].strip() for c in creators]         
        abstract=s.find("meta",{"name":"dc.Description"}) or ''
        keywords=s.find("meta",{"name":"keywords"}) or ''
        url=s.find("meta",{"name":"dc.Source"})['content']        
        if abstract is not '':
            abstract= abstract['content']
        if keywords is not '' : 
            keywords= keywords['content']
    
    if publisher=='taf':
        s = bs(html, 'lxml').find('head')
        year=int(s.find("meta",{"name":"og:description"})['content'][1:5])
        vol=s.find("meta",{"name":"pbContext"})['content'].split('.i')[1].split(';')[0].lstrip('0')
        iss=s.find("meta",{"name":"pbContext"})['content'].split('.v')[1].split('.i')[0].lstrip('0')
        title=s.find("meta",{"name":"dc.Title"})['content']
        creators=s.findAll("meta",{"name":"dc.Creator"});names=[c['content'].strip() for c in creators]
        abstract=s.find("meta",{"name":"dc.Description"}) or ''
        keywords=s.find("meta",{"name":"keywords"}) or ''
        doi=find_between(s.find("meta",{"name":"pbContext"})['content']+';','article:article:',';').lower()
        url='http://www.tandfonline.com/doi/'+doi
        names=[' '.join(name.strip().split()) for name in names]
        if keywords is not '':
            keywords=keywords['content']
        if abstract is not '':
            abstract=abstract['content']


    if publisher=='wly':
        s = bs(html, 'lxml').find('head')
        s_body=bs(html, 'lxml').find('body')
        year=s.find('meta',{'name':'citation_publication_date'})['content'][:4]
        vol= s.find('meta',{'name':'citation_volume'})['content']
        iss= s.find('meta',{'name':'citation_issue'})['content']
        title=s.find('meta',{'property':'og:title'})['content']
        creators=s.findAll('meta',{'name':'citation_author'}); names=[c['content'].strip() for c in creators]
        abstract=s_body.find('div',{'class':'article-section__content mainAbstract'}) or ''
        keywords_html=s.findAll('meta',{'name':'citation_keywords'})
        keywords=', '.join([k['content'] for k in keywords_html])
        url=s.find('meta',{'name':'citation_abstract_html_url'})['content']
        if abstract is not '':
            abstract=abstract.find('p').text

    article={
        'source'   : journal_name,
        'date'     : year,
        'volume'   : vol,
        'issue'    : iss,
        'title'    : title,
        'creator'  : names,
        'abstract' : abstract,        
        'keywords' : keywords,
        'doi'      : doi,
        'url'      : url        
    }
    return article

In [33]:
def src2redif(redif_folder,redif_file):
    global article,f
    folder=base_folder+"/htmls"
    files=os.listdir(folder);print len(files)
    l=[]
    i=0
    try:
        os.remove(redif_file)
    except OSError:
        pass
    for f in files:
        i=i+1
        if 'html' in f:
            if i % 100 ==0:
                print i,
            html=open(folder+'/'+f).read()
            article=parse(html,publisher,journal_name)
            with io.open(redif_file, 'a', encoding="utf-8") as f:
                f.write(unicode('Template-Type: ReDIF-Article 1.0\n'))
                f.write(unicode('Title: '+article['title']+'\n'))
                for creator in article['creator']:
                    f.write(unicode('Author-Name: '+creator+'\n'))
                f.write(unicode('Abstract: '+article['abstract']+'\n'))
                f.write(unicode('Year: '+str(article['date'])+'\n'))
                f.write(unicode('Volume: '+article['volume']+'\n'))
                f.write(unicode('Issue: '+article['issue']+'\n'))
                f.write(unicode('Keywords: '+article['keywords']+'\n'))
                f.write(unicode('File-url: '+article['url']+'\n'))
                f.write(unicode('Journal: '+article['source']+'\n'))            
                f.write(unicode('DOI: '+article['doi']+'\n'))            
                f.write(unicode('\n'))            
    print
    return (redif_file)

In [34]:
redif_base_folder='redif/src/'
    
with open('../OpenScience/subject_journals.json') as data_file:    
    subject_journals = json.load(data_file)

journals=[]
for subject in subject_journals:
    journals=journals+(subject['journals'])
    
for journal in journals:    
    jname=journal['jname']
    journal_name=journal['journal']
    data=journal['data']
    for type_pub in data:   
        data_type=type_pub['type']
        publisher=type_pub['publisher']
        if data_type=='parse' and jname=='jrnmis':# and publisher in ['ama','taf','wly','sag']:
            redif_folder=redif_base_folder+jname+'/'
            if not os.path.exists(redif_folder): os.makedirs(redif_folder)
            print publisher,jname,journal_name,
            base_folder=data_type+'/'+publisher+'/'+jname
            redif_file=redif_folder+(publisher+'.redif').replace('/','_')
            #if not os.path.exists(redif_file):
            src2redif(redif_folder,redif_file)

taf jrnmis Journal of Management Information Systems 1346
100 200 300 400 500 600 700 800 900 1000 1100 1200 1300


In [35]:
article

{'abstract': 'More companies have realized that information technology (IT) outsourcing, once viewed as a cost reduction tool, could facilitate and even enable the transformation of their core business processes. The benefits from a potential outsourcing relationship expansion have strategic implications for relational incentive provision. Modeling "information poaching" in IT outsourcing as an incentive problem with contractibility constraints, our analysis shows that this problem could be mitigated in a repeated game where the outsourcing client and the service provider agree on a relational contract. When the two partners share the belief that they can potentially benefit from a future relationship expansion, they are more likely to behave cooperatively during the early stages of their relationship. However, when they disagree about the likelihood of the future relationship expansion, they will have different preferences on a set of otherwise equivalent relational bonus contracts. S